## Exploring stereo A and B detections

In [2]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import json
import shutil
import sys
from pathlib import Path
from tqdm import tqdm

def find_project_root(marker=".gitignore"):
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / marker).exists():
            return parent.resolve()
    raise FileNotFoundError(
        f"Project root marker '{marker}' not found starting from {current}")
    
root = find_project_root()

In [3]:
stereo_a_detections = pd.read_csv(f"{root}/data/stereo_A_detections.csv")
stereo_b_detections = pd.read_csv(f"{root}/data/stereo_B_detections.csv")

In [4]:
stereo_a_detections.head()

,Unnamed: 0,frame_no,A1_x,A1_y,A1_confidence,A1_method,A2_x,A2_y,A2_confidence,A2_method
0,0,1021,795,468,0.808076,YOLO,1023,452,0.502068,YOLO
1,1,1022,813,462,0.400000,LK,1008,447,0.400000,LK
2,2,1023,832,458,0.823879,YOLO,993,443,0.300000,LK
3,3,1024,851,453,0.812145,YOLO,993,443,0.200000,LK
4,4,1025,851,452,0.400000,LK,983,443,0.200000,LK


In [5]:
stereo_b_detections.head()

,Unnamed: 0,frame_no,B1_x,B1_y,B1_confidence,B1_method,B2_x,B2_y,B2_confidence,B2_method
0,0,1061,16,374,0.797673,YOLO,1892,383,0.400000,LK
1,1,1062,71,383,0.832680,YOLO,1892,383,0.300000,LK
2,2,1063,60,382,0.400000,LK,1745,404,0.848748,YOLO
3,3,1064,178,402,0.845032,YOLO,1700,411,0.832230,YOLO
4,4,1065,225,410,0.840000,YOLO,1657,417,0.833924,YOLO


In [6]:
print(len(stereo_a_detections))
print(len(stereo_b_detections))

317
320


In [15]:
# merge the two dataframes sorted by frame_no
# reject rows with nan values in any column
stereo_ab_detections = pd.merge(stereo_a_detections, stereo_b_detections, on='frame_no', how='outer')
stereo_ab_detections = stereo_ab_detections.sort_values(by='frame_no')
print(len(stereo_ab_detections))


556


In [17]:
stereo_ab_detections.head(20)

,Unnamed: 0_x,frame_no,A1_x,A1_y,A1_confidence,A1_method,A2_x,A2_y,A2_confidence,A2_method,Unnamed: 0_y,B1_x,B1_y,B1_confidence,B1_method,B2_x,B2_y,B2_confidence,B2_method
0,0.0,1021,795.0,468.0,0.808076,YOLO,1023.0,452.0,0.502068,YOLO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1022,813.0,462.0,0.400000,LK,1008.0,447.0,0.400000,LK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,1023,832.0,458.0,0.823879,YOLO,993.0,443.0,0.300000,LK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,1024,851.0,453.0,0.812145,YOLO,993.0,443.0,0.200000,LK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,1025,851.0,452.0,0.400000,LK,983.0,443.0,0.200000,LK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,1026,868.0,449.0,0.300000,LK,946.0,432.0,0.659355,YOLO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6.0,1027,896.0,442.0,0.636824,BG,949.0,433.0,0.772284,BG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7.0,1028,896.0,441.0,0.400000,LK,948.0,433.0,0.400000,LK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8.0,1029,956.0,430.0,0.705377,BG,948.0,433.0,0.300000,LK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9.0,1030,956.0,429.0,0.400000,LK,949.0,433.0,0.200000,LK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
